## Import the required libraries

In [340]:
import pandas as pd
import krakenex
from pandas import json_normalize
import datetime as dt
from datetime import date
from datetime import timezone
from datetime import timedelta
import pygsheets
import time
import numpy as np

## Authentitication

In [341]:
k = krakenex.API()
k.load_key('kraken.key')

## Call the positions

In [342]:
current_pf_status = k.query_private('Balance')
current_pf_status = current_pf_status['result']
current_pf_status = pd.DataFrame.from_dict(current_pf_status, orient='index').reset_index()
current_pf_status = current_pf_status.rename(columns={"index": "crypto", 0: "value"})
current_pf_status['time'] = dt.date.today()
current_pf_status

,crypto,value,time
0,ZEUR,0.0000,2021-04-18
1,XXRP,0.00000000,2021-04-18
2,XXDG,650.28000000,2021-04-18
3,XMLN,2.9263800000,2021-04-18
4,ADA,926.42794400,2021-04-18
5,NANO,21.9957600000,2021-04-18
6,SC,23651.3060300000,2021-04-18
7,TRX,17734.37387000,2021-04-18
8,DOT,41.4684972400,2021-04-18
9,COMP,0.5321800000,2021-04-18


## Get the list of transactions all time

In [343]:
def add_one_month(t):
    one_day = dt.timedelta(days=1)
    one_month_later = t + one_day
    while one_month_later.month == t.month:  # advance to start of next month
        one_month_later += one_day
    target_month = one_month_later.month
    while one_month_later.day < t.day:  # advance to appropriate day
        one_month_later += one_day
        if one_month_later.month != target_month:  # gone too far
            one_month_later -= one_day
            break
    return one_month_later

start = dt.date(2021, 1, 1)

#let's create the master dataset that will list all transactions
transactions_column_names = ['refid', 'time', 'type', 'subtype', 'aclass', 'asset', 'amount', 'fee', 'balance']
transactions_df = pd.DataFrame(columns = transactions_column_names)

while start < dt.date.today():
    
    #create the end date one month later
    end = add_one_month(start)
    #format the dates into unix for the api parameters
    formatted_start = time.mktime(start.timetuple())
    formatted_end = time.mktime(end.timetuple())
    #change the start date for the next loop
    start = end
    
    #let's call the endpoint to have all transaction details in EUR
    req_data = {'aclass': 'ZEUR',
                'start': formatted_start,
                'end': formatted_end
                }
    transactions = k.query_private('Ledgers', req_data)
    
    if transactions.get('error'):
        pass
    
    elif transactions['result']['ledger'] is None:
        pass
    
    else:
        transactions = transactions['result']['ledger']
        transactions = json_normalize(transactions)
        
        #let's pull out the list of all ledger IDs
        ledger_list = list(transactions.columns)
        sep = '.'
        ledger_list = [i.split(sep, 1)[0] for i in ledger_list]
        clean_ledger_list=[]
        for i in ledger_list:
          if i not in clean_ledger_list:
            clean_ledger_list.append(i)
        
        #let's create the temporary dataset that will list all transactions
        temp_df = pd.DataFrame(columns = transactions_column_names)
        
        #let's loop over the list of ledgers to get all the transactions and add them to the master dataframe
        all_transactions = k.query_private('Ledgers', req_data)
        for i in clean_ledger_list:
            transaction_row = all_transactions['result']['ledger'][i]
            transaction_row = json_normalize(transaction_row)
            temp_df = temp_df.append(transaction_row)
    transactions_df = transactions_df.append(temp_df)
transactions_df

,refid,time,type,subtype,aclass,asset,amount,fee,balance
0,TSK6GOI-LZPLB-HL3TLP,1.612030e+09,receive,,currency,XXRP,299.60176000,0.00000000,299.60176000
0,TSK6GOI-LZPLB-HL3TLP,1.612030e+09,spend,,currency,EUR.HOLD,-96.0000,0.0000,0.0000
0,QYTRRLS-QQPVGY-HEZRO3,1.612030e+09,deposit,,currency,EUR.HOLD,100.0000,4.0000,96.0000
0,QYTUHQN-C5XPV4-7GCIW3,1.611550e+09,deposit,,currency,EUR.HOLD,-100.0000,-4.0000,0.0000
0,TSJGHKA-HO6HK-3ZS3DV,1.611527e+09,receive,,currency,XMLN,2.9263800000,0.0000000000,2.9263800000
...,...,...,...,...,...,...,...,...,...
0,TSDP34S-TWBXS-ONQ4VF,1.617444e+09,spend,,currency,EUR.HOLD,-192.2500,0.0000,0.0000
0,QYT75YH-654XRI-UZWBKA,1.617444e+09,deposit,,currency,EUR.HOLD,200.0000,7.7500,192.2500
0,TSLXJOP-ODEZN-MAHRI3,1.617396e+09,receive,,currency,MANA,542.3229600000,0.0000000000,1626.0042500000
0,TSLXJOP-ODEZN-MAHRI3,1.617396e+09,spend,,currency,EUR.HOLD,-481.0000,0.0000,0.0000


## Transform the dataframe to crypto Vs. euro rows

In [344]:
deposit = transactions_df
deposit['fee'] =  deposit['fee'].apply(lambda x: float(x))
deposit = deposit[deposit['type'] == 'deposit']
deposit = deposit.filter(items=['time', 'refid', 'fee', 'amount'])
deposit['time'] = pd.to_datetime(deposit['time'],unit='s')
deposit

,time,refid,fee,amount
0,2021-01-30 18:04:03.146200064,QYTRRLS-QQPVGY-HEZRO3,4.00,100.0000
0,2021-01-25 04:39:23.675200000,QYTUHQN-C5XPV4-7GCIW3,-4.00,-100.0000
0,2021-01-24 22:28:43.057299968,QYTALIZ-3CDRFM-PYTI67,4.00,100.0000
0,2021-01-24 22:23:42.403800064,QYTVDTI-NT42Z2-IPF2Q2,4.00,100.0000
0,2021-01-24 21:38:48.501100032,QYT67L5-2OKDB7-64E3DB,4.00,100.0000
0,2021-01-21 08:29:31.812700160,QYT2UCL-TTHB5O-XB5EY7,5.88,150.0000
0,2021-01-16 11:22:35.174599936,QYTRPRA-NYDDF4-RAI2DK,4.00,100.0000
0,2021-01-13 08:06:05.009700096,QCCD4MT-3KLZU4-IGO5S3,0.00,300.0000
0,2021-01-07 17:40:53.718599936,QYTZU74-B6H7GL-ILTDHN,9.63,250.0000
0,2021-01-07 14:45:10.604100096,QYTIG62-FPT5BQ-ZF7T2S,4.00,100.0000


## Calculate the internal flows

In [345]:
internal_flows = transactions_df
internal_flows = internal_flows[internal_flows['type'] == 'spend']
internal_flows = internal_flows.filter(items=['time', 'refid', 'asset', 'amount'])
internal_flows['time'] = pd.to_datetime(internal_flows['time'],unit='s')
internal_flows = internal_flows[internal_flows['asset'] != 'EUR.HOLD']
internal_flows

,time,refid,asset,amount
0,2021-02-04 18:14:31.747500032,TSQL6E6-VBIBF-S4T5QW,ZEUR,-107.2700
0,2021-02-04 18:07:33.835799808,TSXEVJ7-XHS42-EKJLBS,ZEUR,-100.0000
0,2021-02-04 18:07:00.382999808,TSSUA3C-IH5FV-MY2PWU,ZEUR,-100.0000
0,2021-02-01 14:56:09.868000000,TS7YTE3-72GRT-ZIDMNA,ZEUR,-100.0000
0,2021-02-01 14:45:55.023200000,TSBH3GM-2F5QS-QOETEO,XXRP,-299.60176000
0,2021-03-18 17:00:26.354299904,TSQEGEB-EPVOD-FKHA4E,ZEUR,-200.0000
0,2021-03-15 18:30:33.042099968,TSGFAZB-NXZQE-4ZZ6JM,SC,-14146.3414700000
0,2021-03-13 10:05:27.691600128,TSFW3XM-43UTV-K7T7HT,ZEUR,-300.0000
0,2021-03-13 10:02:06.590500096,TSDHSAF-TKGOB-4FIIHU,ZEUR,-400.0000
0,2021-03-13 08:52:25.094099968,TSRAT5K-YSJ5C-AJN7XE,ZEUR,-400.0000


## Calculate the incoming crypto flows

In [346]:
crypto_positive = transactions_df
crypto_positive = crypto_positive[crypto_positive['type'] == 'receive']
crypto_positive = crypto_positive.filter(items=['time', 'refid', 'asset', 'amount'])
crypto_positive['time'] = pd.to_datetime(crypto_positive['time'],unit='s')
crypto_positive

,time,refid,asset,amount
0,2021-01-30 18:04:04.022599936,TSK6GOI-LZPLB-HL3TLP,XXRP,299.60176000
0,2021-01-24 22:28:48.403700224,TSJGHKA-HO6HK-3ZS3DV,XMLN,2.9263800000
0,2021-01-24 21:38:52.052999936,TSYCEXV-UGPX2-WHLU33,MANA,663.1698200000
0,2021-01-21 08:29:35.913899776,TSCJY65-IR2YI-MQ662X,AAVE,0.9593900000
0,2021-01-16 11:22:40.667099904,TSCST3Q-BFJDP-LMMBYP,COMP,0.5321800000
0,2021-01-07 17:40:56.813199872,TS6JOOY-WIEZS-HORZC4,DOT,28.9504693000
0,2021-01-07 14:45:12.315000064,TSMS4B4-HBJQW-UY6VQI,NANO,21.9957600000
0,2021-01-06 20:27:17.186500096,TSGAUFV-FKGCE-25B6T2,TRX,7822.57677300
0,2021-02-10 12:47:03.640999936,TSUPSKY-HZHJM-QSONBC,DOT,4.9928618900
0,2021-02-10 10:15:35.295699968,TSNJV4Z-XMN6H-47JVZO,ADA,281.69227700


## Create a daily dataframe containing all portfolio transactions regardless of the crypto or currency flow

In [347]:
ongoing_portfolio = crypto_positive.append(internal_flows)
ongoing_portfolio['time'] = ongoing_portfolio['time'].dt.strftime('%Y-%m-%d')
ongoing_portfolio['time'] = pd.to_datetime(ongoing_portfolio['time'], format='%Y-%m-%d')
ongoing_portfolio = ongoing_portfolio.filter(items=['time', 'asset', 'amount'])

# create a range of dates for the merged dataframe
index_of_dates = pd.date_range('2020-01-01', date.today()).to_frame().reset_index(drop=True).rename(columns={0: 'time'})

# create a merged dataframe with date / asset / transactions 
ongoing_portfolio = pd.merge(index_of_dates,ongoing_portfolio,how='left', on='time')
ongoing_portfolio = ongoing_portfolio.rename(columns={'asset': 'crypto'})
ongoing_portfolio

,time,crypto,amount
0,2020-01-01,NaN,NaN
1,2020-01-02,NaN,NaN
2,2020-01-03,NaN,NaN
3,2020-01-04,NaN,NaN
4,2020-01-05,NaN,NaN
...,...,...,...
497,2021-04-14,NaN,NaN
498,2021-04-15,NaN,NaN
499,2021-04-16,XXDG,650.28000000
500,2021-04-17,NaN,NaN


## Add the value of the crypto to the dataframe

In [348]:
#get the list of all crypto that are available in the portfolio
crypto_list = ongoing_portfolio['crypto'].replace({'XMLN':'XMLNZ', 'XXDG':'XDG'}).dropna().unique()
crypto_list = crypto_list + 'EUR'
crypto_list

#set up an empty crypto df
crypto_value_columns = ['time','crypto', 'crypto_value']
crypto_value_df = pd.DataFrame(columns = crypto_value_columns)

for i in crypto_list:
    
    #set up the start date of the loop
    start = dt.date(2021, 1, 1)
    formatted_start = time.mktime(start.timetuple())
    
    #let's call the endpoint to have all transaction details in EUR
    req_data = {'pair': i,
                'interval': '1440',
                'since': formatted_start
                }
    crypto_value = k.query_public('OHLC', req_data)
    
    if crypto_value.get('error'):
        pass
    
    elif crypto_value['result'][i] is None:
        pass
    
    else:
        crypto_value = crypto_value['result'][i]
        crypto_value = pd.DataFrame.from_records(crypto_value)
        crypto_value[0] = pd.to_datetime(crypto_value[0],unit='s')
        crypto_value['crypto'] = i
        crypto_value = crypto_value.rename(columns={0: "time", 1: "crypto_value"})
        crypto_value = crypto_value.filter(items=['time', 'crypto', 'crypto_value'])
    
    crypto_value_df = crypto_value_df.append(crypto_value, ignore_index=True)

crypto_value_df['time'] = pd.to_datetime(crypto_value_df['time'])
crypto_value_df['crypto'] = crypto_value_df['crypto'].apply(lambda x: str(x)[:-3])
crypto_value_df['crypto'] = crypto_value_df['crypto'].replace('XMLNZ', 'XMLN')
crypto_value_df['crypto'] = crypto_value_df['crypto'].replace('XDG', 'XXDG')
crypto_value_df = crypto_value_df.drop(columns=['error'])
crypto_value_df

,time,crypto,crypto_value
0,2021-01-01,TRX,0.021890
1,2021-01-02,TRX,0.021988
2,2021-01-03,TRX,0.021947
3,2021-01-04,TRX,0.023260
4,2021-01-05,TRX,0.025228
...,...,...,...
1448,2021-04-14,XXDG,0.0788826
1449,2021-04-15,XXDG,0.1016633
1450,2021-04-16,XXDG,0.1529947
1451,2021-04-17,XXDG,0.3052828


## Join to the current pf value

In [349]:
last_crypto_value =  current_pf_status.set_index(['time', 'crypto']).join(crypto_value_df.set_index(['time', 'crypto'])).reset_index()
last_crypto_value['value'] = last_crypto_value['value'].apply(lambda x: float(x))
last_crypto_value['crypto_value'] = last_crypto_value['crypto_value'].apply(lambda x: float(x))
last_crypto_value['total_crypto_value'] = last_crypto_value['value']*last_crypto_value['crypto_value']
last_crypto_value = last_crypto_value.replace(np.nan, '')
last_crypto_value = last_crypto_value[last_crypto_value['total_crypto_value'] != '']
last_crypto_value

,time,crypto,value,crypto_value,total_crypto_value
0,2021-04-18,AAVE,0.959390,358.31,343.759031
1,2021-04-18,ADA,926.427944,1.154051,1069.145095
2,2021-04-18,COMP,0.532180,452.07,240.582613
3,2021-04-18,DOT,41.468497,35.4269,1469.100305
5,2021-04-18,GRT,406.662060,1.63535,665.0348
6,2021-04-18,MANA,2472.343730,1.28006,3164.748315
7,2021-04-18,NANO,21.995760,8.801797,193.602214
8,2021-04-18,OCEAN,160.957630,1.3924,224.117404
9,2021-04-18,SC,23651.306030,0.04514,1067.619954
10,2021-04-18,SNX,10.706460,17.101,183.091172


In [350]:
#authorization
gc = pygsheets.authorize(service_file='kraken-reporting-key-sheets.json')

#open the google spreadsheetkraken-credentials
sh = gc.open("Kraken reporting")

#select the first sheet
wks = sh[2]

#update the required sheet
wks.set_dataframe(last_crypto_value,(1,1))

## Join to the ongoing portfolio with the crypto values

In [351]:
final_df = crypto_value_df.set_index(['time', 'crypto']).join(ongoing_portfolio.set_index(['time', 'crypto'])).reset_index()
final_df = final_df.drop(columns=['crypto_value'])
final_df['amount'] = final_df['amount'].apply(lambda x: float(x))
final_df = final_df.replace(np.nan, 0)
final_df['cum_amount'] = final_df.groupby(['crypto']).amount.cumsum()

final_df = final_df[final_df['cum_amount'] != 0]
final_df['time'] = pd.to_datetime(final_df['time'])
final_df = final_df.set_index(['time', 'crypto']).join(crypto_value_df.set_index(['time', 'crypto'])).reset_index()
final_df['cum_amount'] = final_df['cum_amount'].apply(lambda x: float(x))
final_df['crypto_value'] = final_df['crypto_value'].apply(lambda x: float(x))
final_df['cum_amount_eur_value'] = final_df['cum_amount']*final_df['crypto_value']
final_df = final_df.filter(items=['time', 'crypto', 'crypto_value', 'amount', 'cum_amount', 'cum_amount_eur_value'])
final_df

,time,crypto,crypto_value,amount,cum_amount,cum_amount_eur_value
0,2021-01-06,TRX,0.023276,7822.576773,7822.576773,182.078297
1,2021-01-07,DOT,8.165800,28.950469,28.950469,236.403742
2,2021-01-07,NANO,2.072248,21.995760,21.995760,45.580670
3,2021-01-07,TRX,0.024714,0.000000,7822.576773,193.327162
4,2021-01-08,DOT,7.839300,0.000000,28.950469,226.951414
...,...,...,...,...,...,...
1052,2021-04-18,SNX,17.101000,0.000000,10.706460,183.091172
1053,2021-04-18,TRX,0.131001,0.000000,17734.373870,2323.220711
1054,2021-04-18,XMLN,74.804000,0.000000,2.926380,218.904930
1055,2021-04-18,XXDG,0.238459,0.000000,650.280000,155.065053


In [352]:
#authorization
gc = pygsheets.authorize(service_file='kraken-reporting-key-sheets.json')

#open the google spreadsheetkraken-credentials
sh = gc.open("Kraken reporting")

#select the first sheet
wks = sh[3]

#update the required sheet
wks.set_dataframe(final_df,(1,1))

## Send the raw transaction df

In [353]:
raw_dataframe = transactions_df
raw_dataframe['time'] = pd.to_datetime(raw_dataframe['time'],unit='s')
raw_dataframe['time'] = raw_dataframe['time'].dt.strftime('%Y-%m-%d')
raw_dataframe = raw_dataframe.filter(items=['refid', 'time', 'asset', 'amount', 'fee', 'balance'])

#authorization
gc = pygsheets.authorize(service_file='kraken-reporting-key-sheets.json')

#open the google spreadsheetkraken-credentials
sh = gc.open("Kraken reporting")

#select the first sheet
wks = sh[0]

#update the required sheet
wks.set_dataframe(raw_dataframe,(1,1))

## Add aggregated portfolio value to the deposit df 

In [354]:
#create a dataframe with euro spending
eur_spending = deposit
eur_spending = eur_spending.filter(items=['time', 'amount']).rename(columns={"amount": "eur_spent"})
eur_spending['eur_spent'] = eur_spending['eur_spent'].apply(lambda x: float(x))
eur_spending = eur_spending.groupby(['time']).sum().reset_index()
eur_spending['time'] = pd.to_datetime(eur_spending['time']).dt.strftime('%Y-%m-%d')

#create amn aggregated dataframe of crypto values
aggregated_df = final_df
aggregated_df = aggregated_df.groupby(['time']).sum().reset_index()
aggregated_df['time'] = pd.to_datetime(aggregated_df['time']).dt.strftime('%Y-%m-%d')

#combine both dataframes to have both spending and porftolio value
aggregated_df = aggregated_df.set_index('time').join(eur_spending.set_index('time')).reset_index()
aggregated_df['eur_spent'] = aggregated_df['eur_spent'].replace(np.nan, 0)
aggregated_df['eur_spent'] = aggregated_df['eur_spent'].apply(lambda x: float(x))
aggregated_df['cum_eur_spent'] = aggregated_df.eur_spent.cumsum()


In [355]:
#authorization
gc = pygsheets.authorize(service_file='kraken-reporting-key-sheets.json')

#open the google spreadsheetkraken-credentials
sh = gc.open("Kraken reporting")

#select the first sheet
wks = sh[1]

#update the required sheet
wks.set_dataframe(aggregated_df,(1,1))